In [73]:
import numpy as np
import pandas as pd
df = pd.read_csv('diabetes.csv')

In [74]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [76]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [77]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [78]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [79]:
X = scaler.fit_transform(X)

In [80]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [81]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [102]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim = 8))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="rmsprop",loss = "binary_crossentropy",metrics=['accuracy'])

In [103]:
history = model.fit(X_train,Y_train,batch_size=4,epochs  = 100,validation_data=(X_test,Y_test))

Epoch 1/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5932 - loss: 0.6966 - val_accuracy: 0.7403 - val_loss: 0.5504
Epoch 2/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7194 - loss: 0.5504 - val_accuracy: 0.7727 - val_loss: 0.4932
Epoch 3/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7447 - loss: 0.5110 - val_accuracy: 0.7987 - val_loss: 0.4738
Epoch 4/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7485 - loss: 0.4839 - val_accuracy: 0.8052 - val_loss: 0.4663
Epoch 5/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7792 - loss: 0.4398 - val_accuracy: 0.8052 - val_loss: 0.4649
Epoch 6/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7392 - loss: 0.4845 - val_accuracy: 0.7987 - val_loss: 0.4638
Epoch 7/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7311 - loss: 0.4910 - val_accuracy: 0.7987 - val_loss: 0.4649
Epoch 8/100
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7525 - loss: 0.5057 - val_accu

In [104]:
import kerastuner as kt

In [105]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer = hp.Choice('optimizer',values = ['rmsprop','adam','sgd','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [113]:
tuner.search(X_train,Y_train,epochs = 50,validation_data = (X_test,Y_test))

Trial 4 Complete [00h 00m 11s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 39s


In [114]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 02 summary
Hyperparameters:
optimizer: sgd
Score: 0.8181818127632141

Trial 03 summary
Hyperparameters:
optimizer: adam
Score: 0.8181818127632141

Trial 00 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.798701286315918

Trial 01 summary
Hyperparameters:
optimizer: adadelta
Score: 0.4220779240131378


In [115]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [117]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

c:\Users\Ayush Maurya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [124]:
model.fit(X_train,Y_train,epochs = 200,initial_epoch = 51,validation_data = (X_test,Y_test))

Epoch 52/200


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7902 - loss: 0.4348 - val_accuracy: 0.7987 - val_loss: 0.4570
Epoch 53/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8085 - loss: 0.3994 - val_accuracy: 0.7987 - val_loss: 0.4569
Epoch 54/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7903 - loss: 0.4290 - val_accuracy: 0.7987 - val_loss: 0.4570
Epoch 55/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7879 - loss: 0.4099 - val_accuracy: 0.7987 - val_loss: 0.4572
Epoch 56/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7941 - loss: 0.4124 - val_accuracy: 0.7987 - val_loss: 0.4574
Epoch 57/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7931 - loss: 0.4226 - val_accuracy: 0.7987 - val_loss: 0.4577
Epoch 58/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7779 - loss: 0.4460 - val_accuracy: 0.7987 - val_loss: 0.4572
Epoch 59/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7650 - loss: 0.4418 - val_accuracy: 0.7987 - val_lo

In [122]:
print(model.optimizer)


In [138]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',min_value = 8,max_value = 128,step = 8)
    model.add(Dense(units= units,activation='relu',input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [139]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=50)
tuner.search(X_train,Y_train,epochs = 50,validation_data = (X_test,Y_test))

Trial 15 Complete [00h 00m 21s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 03m 15s


In [140]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
units: 8
Score: 0.8051947951316833

Trial 02 summary
Hyperparameters:
units: 80
Score: 0.8051947951316833

Trial 03 summary
Hyperparameters:
units: 56
Score: 0.8051947951316833

Trial 06 summary
Hyperparameters:
units: 96
Score: 0.8051947951316833

Trial 07 summary
Hyperparameters:
units: 104
Score: 0.8051947951316833

Trial 14 summary
Hyperparameters:
units: 40
Score: 0.8051947951316833

Trial 08 summary
Hyperparameters:
units: 48
Score: 0.798701286315918

Trial 11 summary
Hyperparameters:
units: 120
Score: 0.798701286315918

Trial 10 summary
Hyperparameters:
units: 64
Score: 0.7922077775001526

Trial 04 summary
Hyperparameters:
units: 128
Score: 0.7857142686843872


In [141]:
tuner.get_best_hyperparameters()[0].values

{'units': 8}

In [142]:
model = tuner.get_best_models(num_models=10)[0]

c:\Users\Ayush Maurya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [144]:
model.fit(X_train,Y_train,batch_size = 8,initial_epoch=6,epochs = 100,validation_data = (X_test,Y_test))

Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7603 - loss: 0.4934 - val_accuracy: 0.8052 - val_loss: 0.4578
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8094 - loss: 0.4450 - val_accuracy: 0.8052 - val_loss: 0.4561
Epoch 9/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7848 - loss: 0.4685 - val_accuracy: 0.8052 - val_loss: 0.4550
Epoch 10/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7971 - loss: 0.4670 - val_accuracy: 0.8052 - val_loss: 0.4542
Epoch 11/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7833 - loss: 0.4825 - val_accuracy: 0.8052 - val_loss: 0.4535
Epoch 12/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7709 - loss: 0.4631 - val_accuracy: 0.8052 - val_loss: 0.4527
Epoch 13/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7868 - loss: 0.4896 - val_accuracy: 0.8052 - val_loss: 0.4521
Epoch 14/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7859 - loss: 0.4677 - val_accuracy: 0.805

In [154]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(80,activation='relu',input_dim = 8))
    for i in range(hp.Int("num_layers",min_value = 1,max_value = 12)):
        model.add(Dense(80,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
    return model
    


In [155]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)
tuner.search(X_train,Y_train,epochs = 15,validation_data = (X_test,Y_test))

Trial 5 Complete [00h 00m 10s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 38s


In [156]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [157]:
model = tuner.get_best_models(num_models=10)[0]

c:\Users\Ayush Maurya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [158]:
model.fit(X_train,Y_train,batch_size = 8,initial_epoch=6,epochs = 100,validation_data = (X_test,Y_test))

Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7347 - loss: 0.5372 - val_accuracy: 0.7792 - val_loss: 0.4997
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7741 - loss: 0.4911 - val_accuracy: 0.7857 - val_loss: 0.4835
Epoch 9/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7619 - loss: 0.4957 - val_accuracy: 0.7987 - val_loss: 0.4726
Epoch 10/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7467 - loss: 0.4863 - val_accuracy: 0.7987 - val_loss: 0.4665
Epoch 11/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7701 - loss: 0.4726 - val_accuracy: 0.7922 - val_loss: 0.4638
Epoch 12/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7589 - loss: 0.4703 - val_accuracy: 0.7857 - val_loss: 0.4639
Epoch 13/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7619 - loss: 0.4522 - val_accuracy: 0.7857 - val_loss: 0.4654
Epoch 14/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7738 - loss: 0.4625 - val_accuracy: 0.78

In [159]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop, Adadelta

def build_model(hp):
    model = Sequential()
    
    # Input Layer
    model.add(Dense(units=hp.Int('units_1', min_value=16, max_value=128, step=16), 
                    activation=hp.Choice('activation_1', ['relu', 'tanh', 'sigmoid']), 
                    input_dim=8))  # Adjust input_dim as per your dataset
    
    # Hidden Layers (Variable)
    for i in range(hp.Int('num_layers', 1, 4)):  # Vary number of layers from 1 to 4
        model.add(Dense(units=hp.Int(f'units_{i+2}', min_value=16, max_value=128, step=16),
                        activation=hp.Choice(f'activation_{i+2}', ['relu', 'tanh', 'sigmoid'])))
    
    # Output Layer (Binary Classification)
    model.add(Dense(1, activation='sigmoid'))
    
    # Optimizer Selection
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta'])

    if optimizer == 'adam':
        optimizer = Adam()
    elif optimizer == 'rmsprop':
        optimizer = RMSprop()
    else:
        optimizer = Adadelta()

    # Compile Model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize Keras Tuner
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,  # Number of different combinations to try
                        executions_per_trial=1,  # Number of times to train each model
                        directory='my_tuner',
                        project_name='hyperparameter_tuning')


In [ ]:
tuner.search(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test))

# Get best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print best hyperparameters
print("Best Hyperparameters:")
print(f"Number of Layers: {best_hps.get('num_layers')}")
for i in range(1, best_hps.get('num_layers') + 2):  # +2 to include first layer
    print(f"Units in Layer {i}: {best_hps.get(f'units_{i}')}")
    print(f"Activation Function in Layer {i}: {best_hps.get(f'activation_{i}')}")

print(f"Optimizer: {best_hps.get('optimizer')}")


Trial 10 Complete [00h 00m 09s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 01m 14s
Best Hyperparameters:
Number of Layers: 4
Units in Layer 1: 128
Activation Function in Layer 1: relu
Units in Layer 2: 32
Activation Function in Layer 2: sigmoid
Units in Layer 3: 96
Activation Function in Layer 3: relu
Units in Layer 4: 96
Activation Function in Layer 4: sigmoid
Units in Layer 5: 112
Activation Function in Layer 5: tanh
Optimizer: adam


: 